# 1. Título: Aeronaves - Drones cadastrados

## 2. Membros
- Danilo Pimentel de Carvalho Costa - 2016058077
- Dione Jessé Mendes Silva - 2014132130
- Gabriel Lucas Freire Martins e Silva - 2017023250
- Ícaro Kened Torres Neto - 2016077195

## 3. Descrição dos dados
O grupo escolheu o conjunto de dados relacionados ao cadastro de drones, feito pela ANAC. Os dados podem ser encontrados nesta URL: http://dados.gov.br/dataset/aeronaves-drones-cadastrados.

## 4. Diagrama ER
![Diagrama ER](diagrama-er.png "Diagrama ER")

## 5. Diagrama relacional
![Diagrama relacional](diagrama-relacional.png "Diagrama relacional")

## 6. Consultas

In [ ]:
FILE_ENCODING = "ISO-8859-1"
FILE_NAME = "SISANT.csv"
FILE_URL = 'http://sistemas.anac.gov.br/dadosabertos/Aeronaves/drones%20cadastrados/SISANT.csv'

import pandas as pd
import requests
from io import BytesIO, StringIO
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from pathlib import Path
import codecs

print('Step 1 - Reading data')

# If the source file is not present, then request from the data source
if not Path(FILE_NAME).is_file():
    print('Disabling insecure request warnings...')
    requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
    print('Requesting data from source...')
    r = requests.get(FILE_URL, verify=False)
    print('Writing data to file...')
    f = codecs.open(FILE_NAME, "w", FILE_ENCODING)
    f.write(r.content)
    f.close()

print('Reading file content...')
content = Path(FILE_NAME).read_text(encoding = FILE_ENCODING)

print('Creating dataframe...')
df = pd.read_csv(StringIO(content[23:]))

df = df[[
    "Data Validade",
    "Operador",
    "CPF/CNPJ",
    "Tipo Uso",
    "Fabricante",
    "Modelo",
    "Número de serie",
    "Peso máximo de decolagem (Kg)",
    "Cidade-Estado",
    "Ramo de atividade"
]]

df.columns = [
    "licenca_data_validade",
    "operador_nome",
    "operador_cpf_cnpj",
    "licenca_tipo_uso",
    "fabricante_nome",
    "modelo_nome",
    "drone_n_serie",
    "drone_peso_max",
    "operador_cidade_estado",
    "ramo_nome"
]

print('Step 1 - Done!')

In [ ]:
# SQL utilities

DB_NAME = "tp2.db"

import sqlite3

def exec_sql(sql):
    global DB_NAME
    conn = sqlite3.connect(DB_NAME)
    cur = conn.cursor()                               
    cur.execute(sql)
    rows = cur.fetchall()
    conn.commit()
    conn.close()
    return rows

def exec_many_sql(sql, data):
    global DB_NAME
    conn = sqlite3.connect(DB_NAME)
    cur = conn.cursor()                               
    cur.executemany(sql, data)
    rows = cur.fetchall()
    conn.commit()
    conn.close()
    return rows

def toggle_foreign_keys(enable = True):
    exec_sql("PRAGMA foreign_keys = %d" % (1 if enable else 0))
    
def create_table(tbl_name, fields):
    exec_sql("DROP TABLE IF EXISTS %s" % tbl_name)
    exec_sql("CREATE TABLE %s (%s)" % (tbl_name, ','.join(fields)))

def sanitize(value):
    return str(value).replace('"', "").replace("'", "") if (type(value) is str) else str(value)
    
def find_record(tbl_name, condition):
    results = exec_sql("SELECT * FROM %s WHERE %s" % (tbl_name, condition))
    if results:
        return results[0]
    return None

def insert_record(tbl_name, columns, values):
    values = [(sanitize(value)) for value in values]
    exec_sql("INSERT INTO %s (%s) VALUES (%s)" % (tbl_name, ','.join(columns), '"' + '","'.join(values) + '"'))

In [ ]:
print('Step 2 - Creating tables...\n')

print('Creating table "operador"...')
create_table("operador", [
    "operador_id INTEGER PRIMARY KEY AUTOINCREMENT",
    "cpf_cnpj VARCHAR(50)",
    "nome VARCHAR(200)",
    "cidade_estado VARCHAR(50)"
])
print('Done!\n')

print('Creating table "fabricante"...')
create_table("fabricante", [
    "fabricante_id INTEGER PRIMARY KEY AUTOINCREMENT",
    "nome VARCHAR(200)",
])
print('Done!\n')

print('Creating table "modelo"...')
create_table("modelo", [
    "modelo_id INTEGER PRIMARY KEY AUTOINCREMENT",
    "nome VARCHAR(200)",
    "fabricante_id INTEGER",
    "FOREIGN KEY (fabricante_id) REFERENCES fabricante(fabricante_id)"
])
print('Done!\n')

print('Creating table "ramo"...')
create_table("ramo", [
    "ramo_id INTEGER PRIMARY KEY AUTOINCREMENT",
    "nome VARCHAR(200)",
])
print('Done!\n')

print('Creating table "drone"...')
create_table("drone", [
    "n_serie VARCHAR(50) PRIMARY KEY",
    "peso_max FLOAT",
    "ramo_id INTEGER",
    "modelo_id INTEGER",
    "FOREIGN KEY (ramo_id) REFERENCES ramo(ramo_id)",
    "FOREIGN KEY (modelo_id) REFERENCES modelo(modelo_id)"
])
print('Done!\n')

print('Creating table "licenca"...')
create_table("licenca", [
    "operador_id INTEGER",
    "n_serie VARCHAR(50)",
    "tipo_uso VARCHAR(50) NOT NULL",
    "data_validade DATE NOT NULL",
    "PRIMARY KEY (operador_id, n_serie)",
    "FOREIGN KEY (operador_id) REFERENCES operador(operador_id)",
    "FOREIGN KEY (n_serie) REFERENCES drone(n_serie)"
])
print('Done!\n')

print('Step 2 - Done!')

In [ ]:
import re

def create_operador(row):
    if not find_record("operador", 'nome = "%s"' % sanitize(row["operador_nome"])):
        insert_record("operador", ["cpf_cnpj", "nome", "cidade_estado"], [
            row["operador_cpf_cnpj"],
            row["operador_nome"],
            row["operador_cidade_estado"]
        ])
        
def create_fabricante(row):
    if not find_record("fabricante", 'nome = "%s"' % sanitize(row["fabricante_nome"])):
        insert_record("fabricante", ["nome"], [
            row["fabricante_nome"]
        ])

def create_modelo(row):
    fabricante_id = find_record("fabricante", 'nome = "%s"' % sanitize(row["fabricante_nome"]))[0]
    if not find_record("modelo", 'nome = "%s"' % sanitize(row["modelo_nome"])):
        insert_record("modelo", ["nome", "fabricante_id"], [
            row["modelo_nome"],
            fabricante_id
        ])
        
def create_ramo(row):
    if not find_record("ramo", 'nome = "%s"' % sanitize(row["ramo_nome"])):
        insert_record("ramo", ["nome"], [
            row["ramo_nome"]
        ])
        
def create_drone(row):
    ramo_id = find_record("ramo", 'nome = "%s"' % sanitize(row["ramo_nome"]))[0]
    modelo_id = find_record("modelo", 'nome = "%s"' % sanitize(row["modelo_nome"]))[0]
    if not find_record("drone", 'n_serie = "%s"' % sanitize(row["drone_n_serie"])):
        insert_record("drone", ["n_serie", "peso_max", "ramo_id", "modelo_id"], [
            row["drone_n_serie"],
            row["drone_peso_max"],
            ramo_id,
            modelo_id,
        ])
        
def create_licenca(row):
    operador_id = find_record("operador", 'nome = "%s"' % sanitize(row["operador_nome"]))[0]
    n_serie = find_record("drone", 'n_serie = "%s"' % sanitize(row["drone_n_serie"]))[0]
    if not find_record("licenca", 'operador_id = %s AND n_serie = "%s"' % (sanitize(operador_id), sanitize(n_serie))):
        insert_record("licenca", ["operador_id", "n_serie", "tipo_uso", "data_validade"], [
            operador_id,
            n_serie,
            row["licenca_tipo_uso"],
            row["licenca_data_validade"],
        ])

def create_records(row):
    create_operador(row)
    create_fabricante(row)
    create_modelo(row)
    create_ramo(row)
    create_drone(row)
    create_licenca(row)

print('Step 3 - Creating records...')

toggle_foreign_keys(False)

for index, row in df.iterrows():
    create_records(row)

toggle_foreign_keys(True)

print('Step 3 - Done!')

### 6.1 Duas consultas envolvendo seleção e projeção

#### 6.1.1 Consulta 1 (duas versões)

#### 6.1.2 Consulta 2 (duas versões)

### 6.2 Três consultas envolvendo junção de duas relações

#### 6.2.1 Consulta 3 (duas versões)

#### 6.2.2 Consulta 4 (duas versões)

#### 6.2.3 Consulta 5 (duas versões)

### 6.3 Três consultas envolvendo junção de três ou mais relações

#### 6.3.1 Consulta 6 (duas versões)

#### 6.3.2 Consulta 7 (duas versões)

#### 6.3.3 Consulta 8 (duas versões)

### 6.4 Duas consultas envolvendo agregação sobre junção de duas ou mais relações

#### 6.4.1 Consulta 9 (duas versões)

#### 6.4.2 Consulta 10 (duas versões)

## 7. Autoavaliação dos membros